In [2]:
import pandas as pd
import numpy as np



va_activity_loc_assign = pd.read_csv('va_activity_location_assignment.csv.gz', compression='gzip').iloc[:,1:]
va_activity_locations = pd.read_csv('va_activity_locations.csv.gz', compression='gzip').iloc[:,1:]
va_disease_outcome_target = pd.read_csv('va_disease_outcome_target.csv.gz', compression='gzip').iloc[:,1:]
va_disease_outcome_training = pd.read_csv('va_disease_outcome_training.csv.gz', compression='gzip')
va_household = pd.read_csv('va_household.csv.gz', compression = 'gzip').iloc[:,1:]
va_person = pd.read_csv('va_person.csv.gz', compression='gzip')
va_population_network = pd.read_csv('va_population_network.csv.gz', compression='gzip')
va_residence_locations = pd.read_csv('va_residence_locations.csv.gz', compression='gzip').iloc[:,1:]
# va_residence_locations.head()

In [3]:
from graphstats import GraphStats

In [4]:
test = GraphStats(va_activity_loc_assign, va_activity_locations, va_disease_outcome_target, va_disease_outcome_training, va_household, va_person, va_population_network, va_residence_locations)  

#print(va_activity_loc_assign.head(20))
print(test.get_location_durations(4543749))
#test.get_raw_time_with_infected_day(4543889,21)
#test.get_raw_time_with_infected_week(4543889, 50)

{1001002120: 69479, 86307: 13020, 3474: 840}


In [8]:
print(va_disease_outcome_training.head())
print(va_disease_outcome_target.head())

   Unnamed: 0  day      pid state
0     4543749    0  4543749     S
1     4543750    0  4543750     S
2     4543751    0  4543751     S
3     4543752    0  4543752     S
4     4543753    0  4543753     S
       pid  infected
0  4543749         0
1  4543750         0
2  4543751         0
3  4543752         0
4  4543753         0


In [6]:
from preprocess import PreProcessor

p = PreProcessor(va_activity_loc_assign, va_activity_locations, va_disease_outcome_target, va_disease_outcome_training, va_household, va_person, va_population_network, va_residence_locations)
p.get_training_labels(4543889)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])